In [ ]:
import sys
sys.path.append("/Users/kimsan/Desktop/Lecture-Materials/3-1/AIGS538-Deep-Learning/final-project/")

In [ ]:
import torch
import torch.nn as nn
import torchaudio
from transformers import Wav2Vec2ForSequenceClassification, Wav2Vec2Model, Wav2Vec2Processor, AutoProcessor
import soundfile as sf
import librosa
from librosa.feature import melspectrogram
from PIL import Image
from uuid import uuid4
import numpy as np
import sklearn
from matplotlib import pyplot as plt
import pandas as pd
import scipy

from data.dataset import DataSet

%matplotlib inline

In [ ]:
data_df = pd.read_csv("/Users/kimsan/Desktop/Lecture-Materials/3-1/AIGS538-Deep-Learning/final-project/data/birdclef-2023/train_audio/audio_paths.csv")
data_df.head()

In [ ]:
class Wav2VecAudioClassifier(nn.Module):
    def __init__(self):
        super().__init__()

        self.processor = AutoProcessor.from_pretrained("facebook/wav2vec2-base-960h")
        
        self.wav2vec = Wav2Vec2Model.from_pretrained("facebook/wav2vec2-base-960h")
        
        self.pooling = nn.AvgPool1d(768)

        self.dropout = nn.Dropout(0.5)

        self.projection = nn.Linear(206, 264)

        self.relu = nn.ReLU()
    
    def forward(self, audio):
        processed = self.processor(audio, sampling_rate=16000, return_tensors="pt")["input_values"]
        processed = processed.flatten(0, 1)
        context_representation = self.wav2vec(processed)[0]
        pooling_state = self.pooling(context_representation)
        pooling_state = pooling_state.view(pooling_state.shape[0], pooling_state.shape[2], pooling_state.shape[1])
        logits = self.relu(self.projection(pooling_state))

        return logits

In [ ]:
model = Wav2VecAudioClassifier()
optimizer = torch.optim.AdamW(model.parameters(), lr=0.05)
loss_fn = torch.nn.CrossEntropyLoss()

In [ ]:
example_data, sample_rate = librosa.load(data_df.loc[0, "path"])
example_data_label = data_df.loc[0, "label"]

trim_data1 = example_data[: sample_rate*3]
trim_data2 = example_data[3*sample_rate:6*sample_rate]
batch_data = np.array([trim_data1, trim_data2])

one_hot_encodings = {label: index for index, label in enumerate(sorted(list(set(data_df["label"]))))}

example_label_one_hot = torch.zeros(len(set(data_df["label"])))
example_label_one_hot[one_hot_encodings[example_data_label]] = 1
data1_one_hot = example_label_one_hot.clone()
data2_one_hot = example_label_one_hot.clone()

batched_label = torch.stack((data1_one_hot, data2_one_hot))

model.train()
prediction = model.forward(batch_data)
loss = loss_fn(prediction.flatten(1, 2), batched_label)

optimizer.zero_grad()
loss.backward()
optimizer.step()

In [ ]:
params = {
    "batch_size" : 64,
    "shuffle"     : True,
    "num_workers": 6
}
max_epoch = 100

one_hot_encodings = {label: index for index, label in enumerate(sorted(list(set(data_df["label"]))))}

training_set = DataSet(data_df["path"], data_df["label"], one_hot_encodings)
training_generator = torch.utils.data.DataLoader(training_set, **params)

In [ ]:
for epoch in range(max_epoch):
    total_loss = 0.0
    total_correct = 0
    total_samples = 0

    # Training loop
    model.train()
    for audio, labels in training_generator:  # Iterate over your training dataloader
        print(labels)
        # audio = preprocess_audio(audio)
        # labels = preprocess_labels(labels)
        # audio = audio.to("cpu")
        # labels = labels.to("cpu")

        # optimizer.zero_grad()
        # outputs = model(input_values=audio, labels=labels)
        # loss = outputs.loss
        # logits = outputs.logits
        # loss.backward()
        # optimizer.step()

        # total_loss += loss.item()
        # _, predicted_labels = torch.max(logits, dim=1)
        # total_correct += (predicted_labels == labels).sum().item()
        # total_samples += labels.size(0)

    train_loss = total_loss / total_samples
    train_accuracy = total_correct / total_samples
